# Lab 03: TensorFlow vs PyTorch




## TensorFlow Implementation

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # Number of hidden neurons
    tf.keras.layers.Dense(10, activation='softmax')  # Number of output neurons
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Name of the loss function
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")       # Outputs the training time
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3004 - accuracy: 0.9144
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1476 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1067 - accuracy: 0.9684
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0856 - accuracy: 0.9740
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0698 - accuracy: 0.9789
TF Training time: 11.37 seconds
313/313 [==============================] - 0s 894us/step - loss: 0.0901 - accuracy: 0.9721


[0.09011811763048172, 0.972100019454956]

## Convert TensorFlow model to TFLite

In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\vinit\AppData\Local\Temp\tmp9u1hml9a\assets


INFO:tensorflow:Assets written to: C:\Users\vinit\AppData\Local\Temp\tmp9u1hml9a\assets


## PyTorch Implementation

In [4]:
!pip install torchvision

     ---------------------------------------- 1.7/1.7 MB 15.6 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)    # Input and output size
        self.fc2 = nn.Linear(64, 10)    # Input and output size
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

100%|██████████| 9.91M/9.91M [00:01<00:00, 8.81MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 250kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


PyTorch Training time: 27.12 seconds
Test accuracy: 0.9590


## Convert PyTorch model to ONNX

In [9]:
# Installing ONNX
!pip install onnx

  Using cached onnx-1.18.0-cp311-cp311-win_amd64.whl (15.8 MB)



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [11]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalization factor
x_test = x_test / 255.0   # Normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Defining model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Number of neurons and activation
])

# Defining loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3764, Accuracy: 0.0938
Step 100, Loss: 0.4498, Accuracy: 0.7206
Step 200, Loss: 0.3370, Accuracy: 0.7987
Step 300, Loss: 0.4641, Accuracy: 0.8287
Step 400, Loss: 0.2334, Accuracy: 0.8466
Step 500, Loss: 0.3494, Accuracy: 0.8602
Step 600, Loss: 0.1779, Accuracy: 0.8703
Step 700, Loss: 0.2754, Accuracy: 0.8764
Step 800, Loss: 0.2889, Accuracy: 0.8821
Step 900, Loss: 0.4438, Accuracy: 0.8867
Step 1000, Loss: 0.0507, Accuracy: 0.8919
Step 1100, Loss: 0.1385, Accuracy: 0.8949
Step 1200, Loss: 0.1681, Accuracy: 0.8985
Step 1300, Loss: 0.1318, Accuracy: 0.9017
Step 1400, Loss: 0.2969, Accuracy: 0.9038
Step 1500, Loss: 0.2227, Accuracy: 0.9059
Step 1600, Loss: 0.1714, Accuracy: 0.9086
Step 1700, Loss: 0.0311, Accuracy: 0.9113
Step 1800, Loss: 0.3433, Accuracy: 0.9132
Training Accuracy for epoch 1: 0.9147

Epoch 2/5
Step 0, Loss: 0.0423, Accuracy: 1.0000
Step 100, Loss: 0.0710, Accuracy: 0.9579
Step 200, Loss: 0.0516, Accuracy: 0.9512
Step 300, Loss: 0.2131, Accuracy:

## Performance Optimization with Graph Execution using @tf.function

In [12]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalization factor
x_test = x_test / 255.0   # Normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Defining model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    #  Number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Number of neurons and activation
])

# Defining loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compiling the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3757, Accuracy: 0.0938
Step 100, Loss: 0.6098, Accuracy: 0.7277
Step 200, Loss: 0.5080, Accuracy: 0.8033
Step 300, Loss: 0.3916, Accuracy: 0.8341
Step 400, Loss: 0.6318, Accuracy: 0.8519
Step 500, Loss: 0.2983, Accuracy: 0.8646
Step 600, Loss: 0.3261, Accuracy: 0.8716
Step 700, Loss: 0.2038, Accuracy: 0.8787
Step 800, Loss: 0.2351, Accuracy: 0.8837
Step 900, Loss: 0.1710, Accuracy: 0.8895
Step 1000, Loss: 0.5060, Accuracy: 0.8937
Step 1100, Loss: 0.0736, Accuracy: 0.8976
Step 1200, Loss: 0.3479, Accuracy: 0.9006
Step 1300, Loss: 0.1611, Accuracy: 0.9034
Step 1400, Loss: 0.1343, Accuracy: 0.9057
Step 1500, Loss: 0.0496, Accuracy: 0.9086
Step 1600, Loss: 0.0645, Accuracy: 0.9109
Step 1700, Loss: 0.0671, Accuracy: 0.9131
Step 1800, Loss: 0.2054, Accuracy: 0.9153
Training Accuracy for epoch 1: 0.9169

Epoch 2/5
Step 0, Loss: 0.1082, Accuracy: 0.9688
Step 100, Loss: 0.2367, Accuracy: 0.9471
Step 200, Loss: 0.0786, Accuracy: 0.9509
Step 300, Loss: 0.0875, Accuracy: